### 2.2 Compute 'irregular + concave shape' object orientation using Convex Hull + PCA

- The `cv2.convexHull()` function in OpenCV is used to find the **convex hull** of a set of points, 
    - which is the smallest convex shape that fully encloses the given points. 
- It is commonly used in **shape analysis and object detection** to remove concavities and outliers.
```python
    hull = cv2.convexHull(points[, hull[, clockwise[, returnPoints]]])
```
- Parameter :
    - `points` are the contours we pass into. <br>
    - `hull` Output array to store hull indices (used when `returnPoints=False`). <br>
    - `clockwise` : Orientation flag. If it is `True`, the output convex hull is oriented clockwise. Otherwise, it is oriented counter-clockwise. <br>
    - `returnPoints` : If `True`, returns the hull points. If `False`, returns the indices of convex hull points in the original contour. Default: `True`. <br>
    <img src="resource/convexitydefects.jpg" style="width:200px; margin-top:10px;"></img><br><br>


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

print("OpenCV version: " + cv2.__version__)

#### EXAMPLE 1 - Find Object Orientation using PCA + Convex Hull

In [ ]:
# Load and convert the image to grayscale
image = cv2.imread("hands.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 1: Apply simple Binary Thresholding (Inverse Binary Thresholding)
_, binary = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY)

# Step 2: Apply Canny Edge Detection
edges = cv2.Canny(binary, 50, 200)

# Display the images binary and edges
plt.figure(figsize=(20,10))

plt.subplot(1, 2, 1)
plt.imshow(binary, cmap="gray")
plt.title("Binary Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(edges, cmap="gray")
plt.title("Canny Edge Detection")
plt.axis("off")

plt.show()

In [ ]:
# Step 3: Find contours
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Step 4: Calculate minimum area bounding rectangle
output = image.copy()
for contour in contours:
    
    # filter out small contours
    if cv2.contourArea(contour) < 50:
        continue

    # modify contour by applying convex hull
    contour = cv2.convexHull(contour)

    # draw contour
    cv2.drawContours(output, [contour], -1, (0, 255, 0), 1)

    # calculate PCA of the contour
    data_points = np.array(contour.squeeze(), dtype=np.float64)
    mean, eigenvectors = cv2.PCACompute(data_points, mean=None)

    # calculate the center of the contour
    center = tuple(mean[0].astype(int))

    # eigenvectors[0, 0] is the x-component of the first eigenvector
    # eigenvectors[0, 1] is the y-component of the first eigenvector
    # Computes the angle (θ) of the principal axis with respect to the x-axis.
    angle_rad_original = np.arctan2(eigenvectors[0, 1], eigenvectors[0, 0]) # in radians within the range [-π, π]
    angle_rad = (angle_rad_original - np.pi) % (-1 * np.pi) # in radians within the range [0, π]
    angle_rad = abs(angle_rad) # Convert to range [0, π]
    
    # Define the length of the orientation lines
    length = 50

    # Black line for original orientation, always referenced as horizontal line
    cv2.line(output, center, (center[0] + length, center[1]), (0, 0, 0), 2)  

    # Compute the end point for the **rotation axis** (red line)
    end_x = int(center[0] + length * np.cos(angle_rad)) # x = x0 + r * cos(angle)
    end_y = int(center[1] - length * np.sin(angle_rad)) # y = y0 - r * sin(angle)
    cv2.line(output, center, (end_x, end_y), (0, 0, 255), 2)  # Red line for rotation axis

    # Draw angle text near the red line
    angle = np.rad2deg(angle_rad) # in degrees within the range [0, 180]
    text_position = (center[0] - 7, center[1] - 7)
    cv2.putText(output, f"{int(angle)}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, (0, 0, 0), 1, cv2.LINE_AA)

                    
# Show results using Matplotlib
plt.figure(figsize=(20, 6))
plt.imshow(output[:, :, ::-1])  # Convert BGR to RGB
plt.axis("off")
plt.title("Object Orientation with Reference Line")
plt.show()

- ✅ Pros
    - ✔ More stable for concave objects: The Convex Hull removes indentations and irregularities, leading to a more robust orientation estimate.
    - ✔ Reduces noise sensitivity: Small noise points or artifacts outside the object boundary are ignored.
    - ✔ Better alignment with human perception: The convex shape better represents the "true" object orientation.
- ❌ Cons
    - ✖ Less effective for <font color="orange">elongated objects</font>: If the object is already well-represented by PCA, Convex Hull may not add much value.
    - ✖ Inaccurate when convex hull result become <font color="orange">non-elongated shape</font> and Loses shape details.<br><br>
    <img src="resource/plane_output.png" width="50%">


## Source :

- https://docs.opencv.org/4.x/d7/d1d/tutorial_hull.html